# Neural networks and backpropagation

Neural nets are a very powerful class of methods that have become popular in fields like computer vision and natural language processing, where coming up with good features can be challenging. 

While there's a rich mathematical foundation underlying neural nets, in this class we'll focus on one of the big computational ideas that's at the heart of most neural net implementations: backpropagation and automatic differentiation. While these ideas were initially conceived for neural nets, they're now used in many other ways too: libraries like PyMC3 use automatic differentiation to do efficient Bayesian inference; and much more.

In general, automatic differentiation and backpropagation are useful for any problem where the solution involves computing gradients!

### What is a neural net?

Linear regression is a simple but powerful model: to predict a value $y$ from a vector of features $x = (x_1, \ldots, x_k)$, linear regression uses the following:

$$
y = Wx + b
$$

The only way to model complex, nonlinear interactions using linear regression is to define more complex features. But what if we could add another layer of linear regression that computes features for us?

$$
y = W_2(\overbrace{W_1 x + b_1}^\text{features}) + b_2
$$

Unfortunately, this doesn't work because it reduces to a single layer of linear regression. We can simplify this to $y = \big(W_2W_1\big)x + \big(W_2b_1 + b_2\big)$, which is just linear regression written in an unnecessarily complicated way. In order to prevent the simplification to plain old linear regression, we can add a nonlinearity:

$$
y = W_2 f(W_1 x + b_1) + b_2
$$

Here, the nonlinear function $f$ can be anything from a sigmoid (common in the original neural nets) to the simple ReLU (restricted linear unit) function, $f(z) = \max(0, z)$. To make our notation a little simpler, we'll use $\theta$ to denote all our parameters: $\theta = (W_1, W_2, b_1, b_2)$

In order to find the best values of $\theta$, we'll define a loss function $\ell(\theta, y)$ and then use stochastic gradient descent to minimize it.

### Review: Stochastic gradient descent

(Stochastic) gradient descent is a powerful tool that lets us find the minimum of any function, as long as we can compute its gradient. Recall that a gradient is a vector of partial derivatives with respect to each parameter. In the example above, our gradient would be

$$
\nabla_\theta \ell (\theta, y) = 
    \begin{pmatrix}
        \frac{\partial \ell}{\partial W_1}(\theta, y)\\
        \frac{\partial \ell}{\partial W_2}(\theta, y)\\
        \frac{\partial \ell}{\partial b_1}(\theta, y)\\
        \frac{\partial \ell}{\partial b_2}(\theta, y)
    \end{pmatrix}
$$

In gradient descent, we start with an initial estimate for our parameters, $\theta^{(0)}$. We then repeatedly apply the following update to get $\theta^{(1)}, \theta^{(2)}, \ldots$:

$$
\theta^{(t+1)} = \theta^{(t)} - \alpha \nabla_\theta \ell(\theta^{(t)}, y)
$$

Here, $\alpha$ is a learning rate (typically a small positive number), and $y$ is the data we observed. In **stochastic** gradient descent, instead of computing the gradient using all of our data, we only use a random subset of the data at each iteration instead.

### Computing gradients with the chain rule

Backpropagation is an algorithm for efficiently computing derivatives by applying the chain rule. To see how it works, let's consider a very simple loss function involving three variables, $a$, $b$, and $c$:

$$
L(a, b, c) = (a + 3b)c^2
$$

We can compute the partial derivatives with respect to $a$, $b$, and $c$. To make it a little clearer when and where we're using the chain rule, let $q = a+b$ and $r = c^2$.

$$
\begin{align*}
\frac{\partial \ell}{\partial a} &= \frac{\partial \ell}{\partial q}\cdot\frac{\partial q}{\partial a} = c^2 \cdot 1 \\
\frac{\partial \ell}{\partial b} &= \frac{\partial \ell}{\partial q}\cdot\frac{\partial q}{\partial b} = c^2 \cdot 3 \\
\frac{\partial \ell}{\partial c} &= \frac{\partial \ell}{\partial r}\cdot\frac{\partial r}{\partial c} = (a+b) \cdot 2c
\end{align*}
$$

Even in this simple example, we can see that there was some redundant work involved: in doing this computation, we needed to compute $\frac{\partial \ell}{\partial q} = c^2$ twice. In a more complicated expression, the redundant work would become much worse. Backpropagation gives us a way to compute these gradients more efficiently.



### Backpropagation: an example

Instead of representing the computation as an algebraic expression, let's express it as a computation graph: 

![](computation_graph.png)

Given specific numerical values for $a$, $b$, and $c$, backpropagation is an efficient way to compute the loss and the gradient (i.e., all the partial derivatives), with no redundant computation.

We start by computing the loss itself. This involves just doing the computations specified by the graph, denoted by the blue numbers above the arrows:

![](backprop_forward.jpg)

Next, let's notice that when we did the calculations in the previous section to find the gradient, most of our expressions started at the loss, then, using the chain rule, computed partial derivatives with respect to things like $q$ and $r$. Let's try to write these partial derivatives on the graph, and see if we can use them to keep working backwards.

1. First, we'll start with the easiest derivative, the derivative of the loss with respect to itself: $\frac{\partial L}{\partial L}$. This is just 1!

2. Next, we'll compute the derivative of the loss with respect to $q$ (top right branch of the graph). How did we get from $q$ to $L$? We multiplied by 16 (that is, for these specific numbers, $L = 16q$). So, the partial derivative of $L$ with respect to $q$ is just 16. 

3. Continuing along the top part of the graph, now we can compute the derivative with respect to $a$. How did we get from $a$ to $q$? We added 9 (that is, for these specific numbers, $q = a + 9$). So, the partial derivative of $q$ with respect to $a$ is just 1. But we're trying to compute $\frac{\partial L}{\partial a}$, not $\frac{\partial q}{\partial a}$. So, we'll take advantage of the chain rule and multiply by the "derivative so far": that's just $\frac{\partial L}{\partial q}$ = 16. So, our answer is $\frac{\partial L}{\partial a} = 1 \cdot 16 = 16$.

4. Next, we'll look at the $b$ branch of the graph. From similar reasoning to above, the derivative at the output of the "multiply by three" block is just 16. How do we use that to compute the derivative with respect to $b$? To get from $b$ to that value, we multiplied by 3. So, the corresponding term in the chain rule is 3. We multiply that with what we have so far, 16, to get 48.

5. Finally, let's look at the $c$ branch at the bottom of the graph. We'll start by computing the derivative with respect to $r$. Similar to step 2 above, we multiplied $r$ by 11 to get $L$, so that means that the derivative is 11.

6. All we have left is to go through the "square" block. The derivative of its output with respect to its input is two times the input (in other words, $\frac{\partial r}{\partial c} = 2c$). Since the input was 4, that means our new term is 8, and our overall derivative on this branch is $11 \cdot 8 = 88$.

Now we're done! We've computed the derivatives, as shown in this completed graph with the backpropagation intermediate and final results in red below the arrows:

![](backprop_filled.jpg)

### Backpropagation

In general, all we need to successfully run backpropagation is the ability to differentiate every mathematical building block of our loss (don't forget, the loss depends on the prediction). For every building block, we need to know how to compute the forward pass (the mathematical operation, like addition, multiplication, squaring, etc.) and the backward pass (multiplying by the derivative).

Let's see what this looks like in code:

In [23]:
import torch

# Torch tensors are like numpy arrays
a = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
c = torch.tensor(4., requires_grad=True)

In [17]:
q = a + 3 * b
r = c ** 2
print(q, r)

tensor(11., grad_fn=<AddBackward0>) tensor(16., grad_fn=<PowBackward0>)


In [18]:
L = q * r
L

tensor(176., grad_fn=<MulBackward0>)

In [19]:
L.backward()

In [20]:
a.grad

tensor(16.)

In [21]:
b.grad

tensor(48.)

In [22]:
c.grad

tensor(88.)